# Computation for Proposition 3.5

In [1]:
import os
import pandas as pd
from inspect import getsource
from utils import *
if not os.path.exists('data/'):
    os.makedirs('data')

We shall compute the upper bound of $$h-6\log\text{rad}(N) + \log(1728)$$ for $h<e^{31}$.

In [3]:
if not os.path.isfile('data/list_of_f3_3e7.pkl'):
    # the computation needs near to one hour
    list_of_f3_3e7 = list_of_f3(3*pow(10,7))
    pd.to_pickle(list_of_f3_3e7,"data/list_of_f3_3e7.pkl")
else:
    list_of_f3_3e7 = pd.read_pickle("data/list_of_f3_3e7.pkl")

my_primes = []
for p in primes(3*pow(10,7)):
    if p >= 23 and p not in (37,43,67,163):
        my_primes.append(p)
N = len(my_primes)

100%|██████████| 900000002/900000002 [05:00<00:00, 2994675.54it/s]


Let $p_0 < \cdots < p_{N-1}$ the the primes in the list *my_primes*.
For each $0\le t \le N-1$, when $p_t \cdot p_{t+1} > h / \log(2)$,
put $k = 2$, $S=\{p_t, \dots, p_{t+n-1}\}$ for some choice of $2 \le n = |S| \le N+1-t$,
then we can apply Lemma 3.1 $(ii)$ to get the inequality
$$ h \le (6+a_1)\cdot \log\text{rad}(N) + a_2 + \frac{2h}{n},$$
hence [by multiplying $6$, adding $a_1\cdot h$, and dividing $6+a_1$, we have]
$$ h \le 6 \cdot \log\text{rad}(N) + \frac{a_1+12n^{-1}}{6+a_1}\cdot h +\frac{6a_2}{6+a_1}.$$


Let $h_1 = p_s*p_{s+1}*\log(2)$, $1 < h_0 < h_1$. 
Then for any $h_0\le h < h_1$, we have 
$$h - 6\cdot \log\text{rad}(N) + \log(1728) \le a_h\cdot \sqrt{h\log(h)},$$
where we can take 
$$a_h = \frac{a_1+12n^{-1}}{6+a_1}\cdot\sqrt{\frac{h_1}{\log(h_1)}}
+(\frac{6a_2}{6+a_1}+\log(1728))\cdot {\frac{1}{h_0\log(h_0)}}.$$

The following code verifies the inequality
$$h - 6\cdot \log\text{rad}(N) + \log(1728) \le 68 \cdot \sqrt{h\log(h)}$$
for $350 \le h \le e^{31.5}$.

In [4]:
list_of_diff_info = []
choice_of_t = [0]
tmp = 0
for t in range(600000):
    if t - tmp < t/20 and t>100:
        continue
    choice_of_t.append(t)
    tmp = t

h0 = 0
h1 = 100
for t in tqdm(choice_of_t):
    p0 = my_primes[t]
    p1 = my_primes[t+1]
    n = int( 1/3 * p0 / sqrt(log(p0))-2)
    if t + n > N:
        print("Error!")
    a1 = 0 # avg of O(1/p)
    a2 = 0 # avg of log-volume
    for i in range(n):
        p = my_primes[t+i]
        a1 += (119*p+355)/(5*p*p+5*p-12)
        a2 += list_of_f3_3e7[p] # -------------------------------------
    a1 = a1 / n
    a2 = 6 * a2 / n
    h0 = h1
    h1 = p0*p1*log(2)
    if h0 > 1000:
        hl, hr = h0, h1
        diff = (a1+12/n)/(6+a1)*sqrt(hr/log(hr)) + (6*a2/(6+a1)+log(1728))/sqrt(hl*log(hl))
        list_of_diff_info.append([hl,hr,diff])
    else:
        for i in range(100):
            hl = h0 + (h1-h0)*i/100
            hr = h0 + (h1-h0)*(i+1)/100
            diff = (a1+12/n)/(6+a1)*sqrt(hr/log(hr)) + (6*a2/(6+a1)+log(1728))/sqrt(hl*log(hl))
            if hr < 350:
                continue
            list_of_diff_info.append([hl,hr,diff])

print(list_of_diff_info)


100%|██████████| 269/269 [00:06<00:00, 44.43it/s] 

[[346.3838352147688, 350.0071269091036, 60.49805168078858], [350.0071269091036, 353.63041860343844, 60.21065050372732], [353.63041860343844, 357.2537102977733, 59.92836152969852], [357.2537102977733, 360.87700199210815, 59.651046406203236], [360.87700199210815, 364.50029368644294, 59.37857187080942], [364.50029368644294, 368.1235853807778, 59.110809514981376], [368.1235853807778, 371.74687707511265, 58.847635561095814], [371.74687707511265, 375.3701687694475, 58.5889306517863], [375.3701687694475, 378.9934604637823, 58.3345796508212], [378.9934604637823, 382.61675215811715, 58.084471454778964], [382.61675215811715, 386.24004385245195, 57.83849881483788], [386.24004385245195, 389.8633355467868, 57.596558168046904], [389.8633355467868, 393.48662724112165, 57.35854947748911], [393.48662724112165, 397.1099189354565, 57.12437608079121], [397.1099189354565, 400.7332106297913, 56.89394454647062], [400.7332106297913, 404.35650232412615, 56.66716453764721], [404.35650232412615, 407.979794018461

In [8]:
# check whether $h\le e^{31}$ is covered
print("Is the case $h\le e^\{31\}$ covered? ",log(list_of_diff_info[-1][1])>31.5)
# check that $\alpha_h < 68$
flag = True
for diff_info in list_of_diff_info:
    if diff_info[2]  > 66:
        flag = False
        print(diff)
print(flag)

Is the case $h\le e^\{31\}$ covered?  True
True
